In [1]:
# open csv from particiant_scores/Human Evaluation Score.csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import argparse
from statsmodels.stats.inter_rater import fleiss_kappa
from krippendorff import alpha as krippendorff_alpha
from cerebras.cloud.sdk import Cerebras
from logger import logger

os.chdir('/home/tahlilmahfuz/RAG_Implementation/Evaluation/participant_scores')

,reviewer_id,review,system_response,system_response_accuracy,system_response_grammatical_correctness,system_response_Relevancy,system_response_specificity,llm_baseline_response,llm_baseline_response_accuracy,llm_baseline_response_grammatical_correctness,llm_baseline_response_relevancy,llm_baseline_response_specificity
0,200042150,Great app. Easy to navigate and understand.,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.0,5.0,5.0,4.0,Thank you so much for your kind words! We're t...,5.0,5.0,5.0,5.0
1,200042150,"Wow it very good, very helpful I like this so ...",Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,5.0,4.0,4.0,5.0,Thank you so much for your kind words! 🌟 We’re...,4.0,4.0,4.0,4.0
2,200042150,Very useful to me. Very greatfull all admins a...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,5.0,5.0,5.0,5.0,**السلام عليكم ورحمة الله وبركاته** \n\nThank...,3.0,3.5,4.0,4.5
3,200042150,It's a very wonderful islamic app which has bo...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.5,5.0,4.5,5.0,Thank you so much for your wonderful feedback!...,5.0,5.0,4.0,4.0
4,200042150,Masha Allah Tabarakallah to whoever developed ...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.5,5.0,5.0,4.5,**Assalamu Alaykum wa Rahmatullahi wa Barakatu...,5.0,4.5,5.0,4.5
5,200042150,Very good for understanding Quran,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,5.0,4.5,4.0,4.0,Thank you for your kind words! We're delighted...,4.5,4.5,5.0,5.0
6,200042150,"Best aap of Al Quran, it has word by word tran...",Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.0,4.0,4.0,4.0,**Assalamu Alaikum wa Rahmatullahi wa Barakatu...,5.0,5.0,5.0,5.0
7,200042150,Would like to have options like audio translat...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.5,5.0,5.0,5.0,Thank you for sharing your suggestion! We appr...,5.0,4.5,4.0,3.5
8,200042150,Easy to understand with word by word meaning i...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.0,4.0,4.5,4.5,Thank you for your kind words! 🌙 We're delight...,5.0,4.0,4.5,2.5
9,200042150,Wonderful app for anyone interested in learnin...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,5.0,5.0,5.0,5.0,Thank you so much for your heartfelt review! ...,5.0,4.5,5.0,5.0


# LLM As A Judge

In [59]:
# open csv from particiant_scores/Human Evaluation Score.csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import argparse
from statsmodels.stats.inter_rater import fleiss_kappa
from krippendorff import alpha as krippendorff_alpha
from cerebras.cloud.sdk import Cerebras
from logger import logger

# Load the CSV file
df = pd.read_csv('Only Human Evaluation Scores.csv')
df.head()

,reviewer_id,review,response_1,accuracy,grammatical_correctness,relevancy,specificity
0,200042150,Great app. Easy to navigate and understand.,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.0,5.0,5.0,5.0
1,200042150,"Wow it very good, very helpful I like this so ...",Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,5.0,4.0,4.0,5.0
2,200042150,Very useful to me. Very greatfull all admins a...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,5.0,5.0,5.0,5.0
3,200042150,It's a very wonderful islamic app which has bo...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.5,5.0,4.5,5.0
4,200042150,Masha Allah Tabarakallah to whoever developed ...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.5,5.0,5.0,4.5


In [60]:
# Load all available API keys
api_keys = [os.getenv(f"CEREBRAS_API_KEY_{i}") for i in range(7)]

api_keys = [key for key in api_keys if key]
print(f"Found {len(api_keys)} Cerebras API keys.")

# print(api_keys)
if not api_keys:
    raise ValueError("No Cerebras API keys found in environment variables")

current_api_key_index = 0
client = Cerebras(api_key=api_keys[current_api_key_index])
model = "gpt-oss-120b"
print(api_keys)

Found 6 Cerebras API keys.
[09/Sep/2025 08:28:27] INFO - Retrying request to /v1/tcp_warming in 0.472719 seconds
[09/Sep/2025 08:28:28] INFO - HTTP Request: GET https://api.cerebras.ai/v1/tcp_warming "HTTP/1.1 200 OK"
['csk-x2kewhvre2y3cvt859wckjvh66wcn3n3k3nwcn5n4kem8exc', 'csk-pjfmp5jn93rmvmn2hcprvfjd5ck59fcfxevtpd3c4ycvvktx', 'csk-n328539mn5h84f22jyw9n3r2vvwypd6hwy8262wmkftnke4f', 'csk-nhh664njyv366pvyyn8v8h42jhncxkhw5mvmk8pdvt5yfdvk', 'csk-edvj62mc95848jm8dwmp3e2kkc2dn5xmyx4t8n8c32ntec8w', 'csk-24tkwkcyjfjy4vmynjw6j45d3m53rvjpw2chmyfwwk5fytm8']


In [61]:
def cerebras_invoke(prompt: str, model="gpt-oss-120b") -> str:
    global client, current_api_key_index, api_keys
    while True:
        try:
            response = client.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model=model,
                temperature=0.4
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            error_msg = e.__str__()
            if "request_quota_exceeded" in error_msg:
                logger.error(f"Request quota exceeded for key serial {current_api_key_index}.")
                current_api_key_index += 1
                logger.info(f"Switching to API key serial {current_api_key_index}.")
            elif "token_quota_exceeded" in error_msg:
                logger.error(f"Token quota exceeded for key serial {current_api_key_index}. Please check your API key limits.")
                api_keys.pop(current_api_key_index)
            else:
                logger.info(f"Switching to API key serial {current_api_key_index} due to an error.")
                logger.error(f"An error occurred: {error_msg}")
                raise e

            if current_api_key_index >= len(api_keys):
                current_api_key_index = 0
            if len(api_keys) == 0:
                raise ValueError("All Cerebras API keys have exceeded their quotas or are invalid.")
            client = Cerebras(api_key=api_keys[current_api_key_index])

In [62]:
import pandas as pd

# Define the LLM judge prompt template
def build_llm_judge_prompt(review, response):
    return f"""
You are an expert evaluator for Islamic app review responses. Please rate the following response to a user review on a scale of 1 (poor) to 5 (excellent) for each criterion below. Only provide the number for each criterion, nothing else.

Criteria:
1. Accuracy: Are the facts, etiquette, and cultural references correct??
2. Grammatical Correctness: Is the writing fluent, grammatical, and well-structured?
3. Relevancy: Does the response actually address the review content?
4. Application Specificity: Does it stay relevant to the Islamic app (not a generic answer)?

User Review:
{review}

Response:
{response}

Format your answer as:
Accuracy: <1-5>
Grammatical Correctness: <1-5>
Relevancy: <1-5>
Application Specificity: <1-5>
"""

# Prepare columns for LLM ratings
df['llm_accuracy'] = None
df['llm_grammatical_correctness'] = None
df['llm_relevancy'] = None
df['llm_specificity'] = None

for idx, row in df.iterrows():
    prompt = build_llm_judge_prompt(row['review'], row['response_1'])
    llm_output = cerebras_invoke(prompt)
    # Parse the LLM output (expects lines like "Accuracy: 5")
    for line in llm_output.splitlines():
        if "Accuracy:" in line:
            df.at[idx, 'llm_accuracy'] = int(line.split(":")[1].strip())
        elif "Grammatical Correctness:" in line:
            df.at[idx, 'llm_grammatical_correctness'] = int(line.split(":")[1].strip())
        elif "Relevancy:" in line:
            df.at[idx, 'llm_relevancy'] = int(line.split(":")[1].strip())
        elif "Application Specificity:" in line:
            df.at[idx, 'llm_specificity'] = int(line.split(":")[1].strip())

[09/Sep/2025 08:28:32] INFO - HTTP Request: POST https://api.cerebras.ai/v1/chat/completions "HTTP/1.1 200 OK"
[09/Sep/2025 08:28:32] INFO - HTTP Request: POST https://api.cerebras.ai/v1/chat/completions "HTTP/1.1 200 OK"
[09/Sep/2025 08:28:33] INFO - HTTP Request: POST https://api.cerebras.ai/v1/chat/completions "HTTP/1.1 200 OK"
[09/Sep/2025 08:28:34] INFO - HTTP Request: POST https://api.cerebras.ai/v1/chat/completions "HTTP/1.1 200 OK"
[09/Sep/2025 08:28:34] INFO - HTTP Request: POST https://api.cerebras.ai/v1/chat/completions "HTTP/1.1 200 OK"
[09/Sep/2025 08:28:35] INFO - HTTP Request: POST https://api.cerebras.ai/v1/chat/completions "HTTP/1.1 200 OK"
[09/Sep/2025 08:28:36] INFO - HTTP Request: POST https://api.cerebras.ai/v1/chat/completions "HTTP/1.1 200 OK"
[09/Sep/2025 08:28:37] INFO - HTTP Request: POST https://api.cerebras.ai/v1/chat/completions "HTTP/1.1 200 OK"
[09/Sep/2025 08:28:37] INFO - HTTP Request: POST https://api.cerebras.ai/v1/chat/completions "HTTP/1.1 200 OK"
[

In [63]:
# Save the DataFrame with LLM ratings
df.to_csv('Human Evaluation Scores with LLM.csv', index=False)

In [64]:
# Load the CSV directly (no need for open())
df = pd.read_csv('./Human Evaluation Scores with LLM.csv')
df

,reviewer_id,review,response_1,accuracy,grammatical_correctness,relevancy,specificity,llm_accuracy,llm_grammatical_correctness,llm_relevancy,llm_specificity
0,200042150,Great app. Easy to navigate and understand.,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.0,5.0,5.0,5.0,5,4,5,5
1,200042150,"Wow it very good, very helpful I like this so ...",Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,5.0,4.0,4.0,5.0,5,5,5,5
2,200042150,Very useful to me. Very greatfull all admins a...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,5.0,5.0,5.0,5.0,4,4,5,5
3,200042150,It's a very wonderful islamic app which has bo...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.5,5.0,4.5,5.0,4,4,5,5
4,200042150,Masha Allah Tabarakallah to whoever developed ...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.5,5.0,5.0,4.5,5,4,5,5
...,...,...,...,...,...,...,...,...,...,...,...
65,200042163,Alhamdulillah after searching a lot finally i ...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,5.0,4.0,4.0,5.0,5,4,5,5
66,200042163,The reminder is good and always remind me to r...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.0,4.0,4.0,4.0,5,5,5,5
67,200042163,Great Apps if this provide another khat beside...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.0,4.0,4.0,3.0,5,5,5,5
68,200042163,Probably the best Qur'an App I've ever used. G...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,5.0,4.0,3.0,3.0,5,4,5,5


In [65]:
import pandas as pd
import numpy as np
from krippendorff import alpha as krippendorff_alpha
from sklearn.metrics import cohen_kappa_score

# Load the CSV with both human and LLM scores
df = pd.read_csv('Human Evaluation Scores with LLM.csv')

categories = ['accuracy', 'grammatical_correctness', 'relevancy', 'specificity']
llm_categories = ['llm_accuracy', 'llm_grammatical_correctness', 'llm_relevancy', 'llm_specificity']

results = []

for cat, llm_cat in zip(categories, llm_categories):
    # Build a matrix: rows=items, columns=raters (human, LLM)
    ratings = pd.DataFrame({
        'human': df[cat].values,
        'llm': df[llm_cat].values
    })
    # Krippendorff expects shape (raters, items)
    ratings_matrix = ratings.T.values

    # Krippendorff's alpha (ordinal)
    ka = krippendorff_alpha(ratings_matrix, level_of_measurement='ordinal')
    # Convert to integers for Cohen's Kappa
    ratings_int = ratings.round().astype(int)
    # Krippendorff expects shape (raters, items)
    ratings_matrix = ratings.T.values
    ka = krippendorff_alpha(ratings_matrix, level_of_measurement='ordinal')
    fk = cohen_kappa_score(ratings_int['human'], ratings_int['llm'])
    human_mean, human_std = np.mean(ratings['human']), np.std(ratings['human'])
    llm_mean, llm_std = np.mean(ratings['llm']), np.std(ratings['llm'])
    results.append((cat.title().replace('_', ' '), fk, ka, human_mean, human_std, llm_mean, llm_std))
# Create a DataFrame with the results
results_df = pd.DataFrame(
    results,
    columns=[
        'Category', "Cohen's Kappa", 'Krippendorff Alpha', 'Human Mean', 'Human Std', 'LLM Mean', 'LLM Std'
    ]
)

print("|Category|Cohen's Kappa|Krippendorff Alpha|Human Mean ± Std|LLM Mean ± Std|")
print("|---|---|---|---|---|")
for _, row in results_df.iterrows():
    print(f"| {row['Category']} | {row['Cohen\'s Kappa']:.2f} | {row['Krippendorff Alpha']:.2f} | {row['Human Mean']:.2f} ± {row['Human Std']:.2f} | {row['LLM Mean']:.2f} ± {row['LLM Std']:.2f} |")

results_df

|Category|Cohen's Kappa|Krippendorff Alpha|Human Mean ± Std|LLM Mean ± Std|
|---|---|---|---|---|
| Accuracy | 0.03 | -0.01 | 4.67 ± 0.48 | 4.80 ± 0.50 |
| Grammatical Correctness | -0.25 | -0.26 | 4.49 ± 0.50 | 4.39 ± 0.52 |
| Relevancy | 0.01 | -0.16 | 4.45 ± 0.63 | 4.93 ± 0.26 |
| Specificity | 0.04 | -0.21 | 4.50 ± 0.62 | 4.99 ± 0.12 |


,Category,Cohen's Kappa,Krippendorff Alpha,Human Mean,Human Std,LLM Mean,LLM Std
0,Accuracy,0.032577,-0.006708,4.671429,0.477237,4.800000,0.495696
1,Grammatical Correctness,-0.245955,-0.260003,4.492857,0.496364,4.385714,0.515277
2,Relevancy,0.012097,-0.163234,4.450000,0.633302,4.928571,0.257539
3,Specificity,0.038021,-0.207492,4.500000,0.621059,4.985714,0.118666


# Fleiss Kappa and Kripendoffs alpha(Mukit,Sadaf,Shoyeb,Tanzim)

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.inter_rater import fleiss_kappa
from krippendorff import alpha as krippendorff_alpha

# Load the CSV
df = pd.read_csv('Inter-rater-reliability.csv')

# Extract all reviewer columns for each criterion
criteria = ['accuracy', 'grammatical_correctness', 'relevancy', 'specificity']
reviewer_ids = ['70', '63', '57']

for crit in criteria:
    # Collect ratings for this criterion from all reviewers
    cols = [f"{rid}_{crit}" for rid in reviewer_ids]
    ratings = df[cols].values.T  # shape: (n_raters, n_items)
    
    # For Fleiss' Kappa, we need a matrix of shape (n_items, n_categories)
    # We'll use categories 1-5, count how many raters gave each score per item
    n_categories = 5
    fleiss_matrix = np.zeros((ratings.shape[1], n_categories), dtype=int)
    for i, item_ratings in enumerate(ratings.T):
        for score in item_ratings:
            if not np.isnan(score):
                fleiss_matrix[i, int(score)-1] += 1

    fk = fleiss_kappa(fleiss_matrix)
    ka = krippendorff_alpha(ratings, level_of_measurement='ordinal')
    print(f"{crit.title()}: Fleiss' Kappa = {fk:.2f}, Krippendorff's Alpha = {ka:.2f}")

Accuracy: Fleiss' Kappa = 0.57, Krippendorff's Alpha = 0.68
Grammatical_Correctness: Fleiss' Kappa = 0.52, Krippendorff's Alpha = 0.54
Relevancy: Fleiss' Kappa = 0.47, Krippendorff's Alpha = 0.57
Specificity: Fleiss' Kappa = 0.79, Krippendorff's Alpha = 0.86


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.inter_rater import fleiss_kappa
from krippendorff import alpha as krippendorff_alpha

# Load the CSV
df = pd.read_csv('Inter-rater-reliability.csv')

# Extract all reviewer columns for each criterion
criteria = ['accuracy', 'grammatical_correctness', 'relevancy', 'specificity']
reviewer_ids = ['70', '63', '57']

results = []
for crit in criteria:
    # Collect ratings for this criterion from all reviewers
    cols = [f"{rid}_{crit}" for rid in reviewer_ids]
    ratings = df[cols].values.T  # shape: (n_raters, n_items)
    
    # For Fleiss' Kappa, we need a matrix of shape (n_items, n_categories)
    n_categories = 5
    fleiss_matrix = np.zeros((ratings.shape[1], n_categories), dtype=int)
    for i, item_ratings in enumerate(ratings.T):
        for score in item_ratings:
            if not np.isnan(score):
                fleiss_matrix[i, int(score)-1] += 1

    fk = fleiss_kappa(fleiss_matrix)
    ka = krippendorff_alpha(ratings, level_of_measurement='ordinal')
    # results.append((crit.title().replace('_', ' '), fk, ka))
     # Calculate mean and std across all raters and items for this criterion
    all_scores = ratings.flatten()
    all_scores = all_scores[~np.isnan(all_scores)]
    mean = np.mean(all_scores)
    std = np.std(all_scores)
    results.append((crit.title().replace('_', ' '), fk, ka, mean, std))

# Print as markdown table
print("|Criterion|Fleiss' Kappa|Krippendorff's Alpha|Mean±Std|")
print("|---|---|---|---|---|")
for row in results:
    print(f"| {row[0]} | {row[1]:.2f} | {row[2]:.2f} | {row[3]:.2f} ± {row[4]:.2f} |")
# create a dataframe of results
results_df = pd.DataFrame(
    results,
    columns=['Criterion', "Fleiss' Kappa", "Krippendorff's Alpha", "Mean", "Std"]
)
results_df

|Criterion|Fleiss' Kappa|Krippendorff's Alpha|Mean±Std|
|---|---|---|---|---|
| Accuracy | 0.57 | 0.68 | 4.50 ± 0.62 |
| Grammatical Correctness | 0.52 | 0.54 | 4.17 ± 0.37 |
| Relevancy | 0.47 | 0.57 | 3.90 ± 0.47 |
| Specificity | 0.79 | 0.86 | 4.23 ± 0.76 |


,Criterion,Fleiss' Kappa,Krippendorff's Alpha,Mean,Std
0,Accuracy,0.567901,0.682716,4.500000,0.619139
1,Grammatical Correctness,0.520000,0.536000,4.166667,0.372678
2,Relevancy,0.473684,0.567367,3.900000,0.472582
3,Specificity,0.790941,0.856347,4.233333,0.760847


# Evaluate System Response and LLM(Baseline) Response

In [53]:
# Route to directory
df = pd.read_csv('Human Evaluation Scores.csv')
df

,reviewer_id,review,system_response,system_response_accuracy,system_response_grammatical_correctness,system_response_Relevancy,system_response_specificity,llm_baseline_response,llm_baseline_response_accuracy,llm_baseline_response_grammatical_correctness,llm_baseline_response_relevancy,llm_baseline_response_specificity
0,200042150,Great app. Easy to navigate and understand.,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.0,5.0,5.0,5.0,Thank you so much for your kind words! We're t...,5.0,5.0,5.0,5.0
1,200042150,"Wow it very good, very helpful I like this so ...",Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,5.0,4.0,4.0,5.0,Thank you so much for your kind words! 🌟 We’re...,4.0,4.0,4.0,4.0
2,200042150,Very useful to me. Very greatfull all admins a...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,5.0,5.0,5.0,5.0,**السلام عليكم ورحمة الله وبركاته** \n\nThank...,3.0,3.0,4.0,4.5
3,200042150,It's a very wonderful islamic app which has bo...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.5,5.0,4.5,5.0,Thank you so much for your wonderful feedback!...,5.0,5.0,4.0,4.0
4,200042150,Masha Allah Tabarakallah to whoever developed ...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.5,5.0,5.0,4.5,**Assalamu Alaykum wa Rahmatullahi wa Barakatu...,5.0,4.0,5.0,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...
65,200042163,Alhamdulillah after searching a lot finally i ...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,5.0,4.0,4.0,5.0,"Alhamdulillah, we’re so glad you found what yo...",4.0,4.0,3.0,3.0
66,200042163,The reminder is good and always remind me to r...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.0,4.0,4.0,4.0,Assalamu alaykum! \n\nThank you so much for sh...,4.0,4.0,3.0,4.0
67,200042163,Great Apps if this provide another khat beside...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,4.0,4.0,4.0,3.0,Thank you for sharing your kind words! 🌟 We're...,4.0,4.0,3.0,3.0
68,200042163,Probably the best Qur'an App I've ever used. G...,Assalamu Alaikum wa Rahmatullahi wa Barakatuh....,5.0,4.0,3.0,3.0,Thank you so much for your kind words! 🌟 We're...,5.0,3.0,3.0,3.0


In [54]:
# Define columns for system and LLM responses
system_cols = [
    'system_response_accuracy',
    'system_response_grammatical_correctness',
    'system_response_Relevancy',
    'system_response_specificity'
]
llm_cols = [
    'llm_baseline_response_accuracy',
    'llm_baseline_response_grammatical_correctness',
    'llm_baseline_response_relevancy',
    'llm_baseline_response_specificity'
]
categories = ['Accuracy', 'Grammatical Correctness', 'Relevancy', 'Specificity']

results = []

for sys_col, llm_col, cat in zip(system_cols, llm_cols, categories):
    sys_mean = df[sys_col].mean()
    sys_std = df[sys_col].std()
    llm_mean = df[llm_col].mean()
    llm_std = df[llm_col].std()
    results.append((cat, sys_mean, sys_std, llm_mean, llm_std))

results_df = pd.DataFrame(
    results,
    columns=['Category', 'System Mean', 'System Std', 'LLM Mean', 'LLM Std']
)

# Print as markdown table
print("|Category|System Mean ± Std|LLM Mean ± Std|")
print("|---|---|---|")
for _, row in results_df.iterrows():
    print(f"| {row['Category']} | {row['System Mean']:.2f} ± {row['System Std']:.2f} | {row['LLM Mean']:.2f} ± {row['LLM Std']:.2f} |")

results_df

|Category|System Mean ± Std|LLM Mean ± Std|
|---|---|---|
| Accuracy | 4.67 ± 0.48 | 4.41 ± 0.65 |
| Grammatical Correctness | 4.49 ± 0.50 | 4.46 ± 0.63 |
| Relevancy | 4.45 ± 0.64 | 4.21 ± 0.78 |
| Specificity | 4.50 ± 0.63 | 3.60 ± 1.21 |


,Category,System Mean,System Std,LLM Mean,LLM Std
0,Accuracy,4.671429,0.480683,4.414286,0.648138
1,Grammatical Correctness,4.492857,0.499948,4.464286,0.627402
2,Relevancy,4.450000,0.637875,4.207143,0.782475
3,Specificity,4.500000,0.625543,3.600000,1.211658


In [55]:
# Define columns for system and LLM responses
system_cols = [
    'system_response_accuracy',
    'system_response_grammatical_correctness',
    'system_response_Relevancy',
    'system_response_specificity'
]
llm_cols = [
    'llm_baseline_response_accuracy',
    'llm_baseline_response_grammatical_correctness',
    'llm_baseline_response_relevancy',
    'llm_baseline_response_specificity'
]
categories = ['Accuracy', 'Grammatical Correctness', 'Relevancy', 'Specificity']

results = []

for sys_col, llm_col, cat in zip(system_cols, llm_cols, categories):
    sys_mean = df[sys_col].mean()
    sys_std = df[sys_col].std()
    llm_mean = df[llm_col].mean()
    llm_std = df[llm_col].std()
    results.append((cat, sys_mean, sys_std, llm_mean, llm_std))

# Calculate overall mean and std for system and LLM (across all criteria and all reviews)
system_all = pd.concat([df[col] for col in system_cols], ignore_index=True)
llm_all = pd.concat([df[col] for col in llm_cols], ignore_index=True)
overall_sys_mean = system_all.mean()
overall_sys_std = system_all.std()
overall_llm_mean = llm_all.mean()
overall_llm_std = llm_all.std()
results.append((
    "Overall",
    overall_sys_mean,
    overall_sys_std,
    overall_llm_mean,
    overall_llm_std
))

results_df = pd.DataFrame(
    results,
    columns=['Category', 'System Mean', 'System Std', 'LLM Mean', 'LLM Std']
)

# Print as markdown table
print("|Category|System Mean ± Std|LLM Mean ± Std|")
print("|---|---|---|")
for _, row in results_df.iterrows():
    print(f"| {row['Category']} | {row['System Mean']:.2f} ± {row['System Std']:.2f} | {row['LLM Mean']:.2f} ± {row['LLM Std']:.2f} |")

results_df

|Category|System Mean ± Std|LLM Mean ± Std|
|---|---|---|
| Accuracy | 4.67 ± 0.48 | 4.41 ± 0.65 |
| Grammatical Correctness | 4.49 ± 0.50 | 4.46 ± 0.63 |
| Relevancy | 4.45 ± 0.64 | 4.21 ± 0.78 |
| Specificity | 4.50 ± 0.63 | 3.60 ± 1.21 |
| Overall | 4.53 ± 0.57 | 4.17 ± 0.91 |


,Category,System Mean,System Std,LLM Mean,LLM Std
0,Accuracy,4.671429,0.480683,4.414286,0.648138
1,Grammatical Correctness,4.492857,0.499948,4.464286,0.627402
2,Relevancy,4.450000,0.637875,4.207143,0.782475
3,Specificity,4.500000,0.625543,3.600000,1.211658
4,Overall,4.528571,0.568818,4.171429,0.913404


In [56]:
# Calculate which system is better overall (across all criteria and reviews)

system_better = 0
llm_better = 0
equal = 0
total = 0

for sys_col, llm_col in zip(system_cols, llm_cols):
    for sys_score, llm_score in zip(df[sys_col], df[llm_col]):
        if pd.isna(sys_score) or pd.isna(llm_score):
            continue
        total += 1
        if sys_score > llm_score:
            system_better += 1
        elif llm_score > sys_score:
            llm_better += 1
        else:
            equal += 1

if total > 0:
    system_pct = 100 * system_better / total
    llm_pct = 100 * llm_better / total
    equal_pct = 100 * equal / total
else:
    system_pct = llm_pct = equal_pct = 0

if system_better > llm_better:
    print(f"System response is better in {system_pct:.1f}% of cases.")
    print(f"LLM response is better in {llm_pct:.1f}% of cases.")
elif llm_better > system_better:
    print(f"LLM response is better in {llm_pct:.1f}% of cases.")
    print(f"System response is better in {system_pct:.1f}% of cases.")
else:
    print(f"Both systems are equally good in {equal_pct:.1f}% of cases.")

print(f"Equal ratings: {equal_pct:.1f}%")

System response is better in 40.7% of cases.
LLM response is better in 18.2% of cases.
Equal ratings: 41.1%


In [57]:
system_cols = [
    'system_response_accuracy',
    'system_response_grammatical_correctness',
    'system_response_Relevancy',
    'system_response_specificity'
]
llm_cols = [
    'llm_baseline_response_accuracy',
    'llm_baseline_response_grammatical_correctness',
    'llm_baseline_response_relevancy',
    'llm_baseline_response_specificity'
]
categories = ['Accuracy', 'Grammatical Correctness', 'Relevancy', 'Specificity']

results = []
max_score = 5  # Assuming the rating scale is 1-5

for sys_col, llm_col, cat in zip(system_cols, llm_cols, categories):
    sys_mean = df[sys_col].mean()
    llm_mean = df[llm_col].mean()
    # Normalize to 0-1
    sys_norm = sys_mean / max_score
    llm_norm = llm_mean / max_score
    # Improvement in percentage
    if llm_norm > 0:
        improvement_pct = 100 * (sys_norm - llm_norm) / llm_norm
    else:
        improvement_pct = float('nan')
    results.append((cat, sys_norm, llm_norm, improvement_pct))

# Calculate overall normalized means and improvement
system_all = pd.concat([df[col] for col in system_cols], ignore_index=True)
llm_all = pd.concat([df[col] for col in llm_cols], ignore_index=True)
overall_sys_norm = system_all.mean() / max_score
overall_llm_norm = llm_all.mean() / max_score
if overall_llm_norm > 0:
    overall_improvement = 100 * (overall_sys_norm - overall_llm_norm) / overall_llm_norm
else:
    overall_improvement = float('nan')
results.append((
    "Overall",
    overall_sys_norm,
    overall_llm_norm,
    overall_improvement
))

results_df = pd.DataFrame(
    results,
    columns=['Category', 'System Normalized Mean', 'LLM Normalized Mean', 'Improvement (%)']
)

# Print as markdown table
print("|Category|System Normalized Mean|LLM Normalized Mean|Improvement (%)|")
print("|---|---|---|---|")
for _, row in results_df.iterrows():
    print(f"| {row['Category']} | {row['System Normalized Mean']:.3f} | {row['LLM Normalized Mean']:.3f} | {row['Improvement (%)']:.2f}% |")

results_df

|Category|System Normalized Mean|LLM Normalized Mean|Improvement (%)|
|---|---|---|---|
| Accuracy | 0.934 | 0.883 | 5.83% |
| Grammatical Correctness | 0.899 | 0.893 | 0.64% |
| Relevancy | 0.890 | 0.841 | 5.77% |
| Specificity | 0.900 | 0.720 | 25.00% |
| Overall | 0.906 | 0.834 | 8.56% |


,Category,System Normalized Mean,LLM Normalized Mean,Improvement (%)
0,Accuracy,0.934286,0.882857,5.825243
1,Grammatical Correctness,0.898571,0.892857,0.640000
2,Relevancy,0.890000,0.841429,5.772496
3,Specificity,0.900000,0.720000,25.000000
4,Overall,0.905714,0.834286,8.561644


In [58]:
system_sum = df['system_response_grammatical_correctness'].sum()
llm_sum = df['llm_baseline_response_grammatical_correctness'].sum()
system_sum, llm_sum

mean_system = df['system_response_grammatical_correctness'].mean()
mean_llm = df['llm_baseline_response_grammatical_correctness'].mean()
print(f"System sum: {system_sum}, mean: {mean_system}")
print(f"LLM sum: {llm_sum}, mean: {mean_llm}")

system_count = df['system_response_grammatical_correctness'].count()
llm_count = df['llm_baseline_response_grammatical_correctness'].count()
print(f"System sum: {system_sum}, mean: {mean_system}, count: {system_count}")
print(f"LLM sum: {llm_sum}, mean: {mean_llm}, count: {llm_count}")

System sum: 314.5, mean: 4.492857142857143
LLM sum: 312.5, mean: 4.464285714285714
System sum: 314.5, mean: 4.492857142857143, count: 70
LLM sum: 312.5, mean: 4.464285714285714, count: 70
